# ===============================================================
# 📘 NOTEBOOK 1: Dataset Creation
# SLSL Sentence-Based Dataset Landmark Extraction Pipeline
# Uses MediaPipe Holistic to extract hand, pose, and lip landmarks
# Dataset is structured in Google Drive by sentence folders
# ===============================================================


In [ ]:
# --- 1: Install Dependencies ---

!pip install mediapipe opencv-python pandas numpy tqdm matplotlib
!apt update && apt install -y ffmpeg

import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import json
from tqdm import tqdm
import shutil
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

print("✅ All dependencies installed and mounted successfully!")

In [ ]:
# --- 2: Folder Setup ---

def create_sentence_based_structure():
    """Create necessary folders in Colab workspace"""
    os.makedirs('/content/raw_videos', exist_ok=True)
    os.makedirs('/content/landmarks_data', exist_ok=True)
    os.makedirs('/content/metadata', exist_ok=True)
    os.makedirs('/content/visualizations', exist_ok=True)
    print("✅ Folder structure ready!")
    print("📁 /content/raw_videos – for input videos")
    print("📁 /content/landmarks_data – for output .npy files")
    print("📁 /content/metadata – for metadata and mapping files")
    print("📁 /content/visualizations – for charts and analysis")

In [ ]:
# --- 3: Copy Dataset from Google Drive ---

def copy_from_drive(drive_path='/content/drive/MyDrive/SLSL_Medical_Dataset'):
    """
    Copy your structured folder from Google Drive into Colab.
    Your Drive should have:
      MyDrive/SLSL_Medical_Dataset/
        ├── මගෙ_අතේ_උණුසුම_වැඩි/
        │   ├── signer_01_rep_01.mp4
        │   └── ...
        ├── මගේ_අතේ_තුවාලයක්_තියෙනවා/
        │   ├── signer_01_rep_01.mp4
        │   └── ...
    """
    
    print(f"📂 Looking for dataset at: {drive_path}")
    
    if os.path.exists(drive_path):
        # Remove existing if any
        if os.path.exists('/content/raw_videos'):
            shutil.rmtree('/content/raw_videos')
        
        shutil.copytree(drive_path, '/content/raw_videos', dirs_exist_ok=True)
        print("✅ Copied entire structure from Google Drive!")
        
        # Count and display structure
        sentences = [f for f in os.listdir('/content/raw_videos') 
                    if os.path.isdir(os.path.join('/content/raw_videos', f))]
        print(f"📁 Found {len(sentences)} sentence folders:")
        
        total_videos = 0
        for sentence in sentences:
            sentence_path = f"/content/raw_videos/{sentence}"
            video_files = [f for f in os.listdir(sentence_path) 
                          if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
            total_videos += len(video_files)
            print(f"   📂 {sentence}: {len(video_files)} videos")
        
        print(f"\n📊 TOTAL: {len(sentences)} sentences, {total_videos} videos")
        return sentences
    else:
        print(f"❌ Folder not found in Google Drive at: {drive_path}")
        print("💡 Available folders in MyDrive:")
        try:
            print(os.listdir('/content/drive/MyDrive'))
        except:
            print("Could not list MyDrive contents")
        return None

In [ ]:
# --- 4: Setup MediaPipe Holistic ---

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def setup_mediapipe():
    """Setup MediaPipe Holistic with balanced accuracy/speed"""
    holistic = mp_holistic.Holistic(
        static_image_mode=False,
        model_complexity=1,  # 0=Lite, 1=Full, 2=Heavy
        smooth_landmarks=True,
        enable_segmentation=False,
        refine_face_landmarks=True,  # Need this for lips
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )
    return holistic


In [ ]:
# --- 5: Extract Landmarks from Frame ---

def extract_frame_landmarks(frame, holistic_model):
    """Extract hand, pose, and lip landmarks from one frame"""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_rgb.flags.writeable = False
    results = holistic_model.process(frame_rgb)

    landmarks_dict = {
        'left_hand': None,
        'right_hand': None,
        'pose': None,
        'face': None,
        'lip_roi': None,
        'timestamp': None
    }

    # --- Left Hand (21 points × 4 values) ---
    if results.left_hand_landmarks:
        left_hand = []
        for landmark in results.left_hand_landmarks.landmark:
            left_hand.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        landmarks_dict['left_hand'] = left_hand

    # --- Right Hand (21 points × 4 values) ---
    if results.right_hand_landmarks:
        right_hand = []
        for landmark in results.right_hand_landmarks.landmark:
            right_hand.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        landmarks_dict['right_hand'] = right_hand

    # --- Pose Upper Body (first 25 points × 4 values) ---
    if results.pose_landmarks:
        pose = []
        upper_body_indices = list(range(25))  # Upper body only
        for i in upper_body_indices:
            landmark = results.pose_landmarks.landmark[i]
            pose.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        landmarks_dict['pose'] = pose

    # --- Face and Lips from Face Mesh ---
    if results.face_landmarks:
        # Key lip landmarks for speech reading
        lip_indices = [
            61, 146, 91, 181, 84, 17, 314, 405, 321, 375,
            78, 95, 88, 178, 87, 14, 317, 402, 318, 324,
            308, 415, 310, 311, 312, 13, 82, 81, 80, 191,
            78, 76, 74, 73, 70, 63, 62, 96, 89, 179,
            86, 15, 316, 403, 319, 325, 307, 414, 309, 313
        ]
        
        lip_landmarks = []
        face_landmarks = []
        
        for i, landmark in enumerate(results.face_landmarks.landmark):
            face_landmarks.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            if i in lip_indices:
                lip_landmarks.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        
        landmarks_dict['face'] = face_landmarks
        landmarks_dict['lip_roi'] = lip_landmarks

    return landmarks_dict


In [ ]:
# --- 6: Process a Single Video ---

def process_single_video(video_path, holistic_model, target_frames=60):
    """Process one video and extract landmarks frame by frame"""
    cap = cv2.VideoCapture(video_path)
    frames_data = []
    frame_count = 0

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"\n📹 Processing: {os.path.basename(video_path)}")
    print(f"   Total frames: {total_frames}, Target: {target_frames}")

    with tqdm(total=min(total_frames, target_frames), desc="   Extracting") as pbar:
        while cap.isOpened() and len(frames_data) < target_frames:
            ret, frame = cap.read()
            if not ret:
                break

            landmarks = extract_frame_landmarks(frame, holistic_model)
            landmarks['timestamp'] = frame_count / fps
            frames_data.append(landmarks)
            frame_count += 1
            pbar.update(1)

    cap.release()
    
    # If we have fewer frames than target, pad with zeros
    if len(frames_data) < target_frames:
        print(f"   ⚠️ Only {len(frames_data)} frames, padding to {target_frames}")
        # Create a zero frame template
        if frames_data:
            zero_frame = {k: (np.zeros_like(v) if v is not None else None) 
                         for k, v in frames_data[0].items()}
            while len(frames_data) < target_frames:
                frames_data.append(zero_frame.copy())
    
    print(f"   ✅ Extracted {len(frames_data)} frames")
    return frames_data


In [ ]:
# --- 7: Process Entire Dataset ---

def process_sentence_based_dataset(raw_videos_root='/content/raw_videos',
                                   output_folder='/content/landmarks_data',
                                   target_frames=60,
                                   test_mode=False):
    """Process all sentence folders in dataset"""
    metadata = []
    
    # Initialize MediaPipe
    holistic = setup_mediapipe()
    print("✅ MediaPipe Holistic initialized")

    print("🔍 Scanning for sentence folders...")
    sentence_folders = [f for f in os.listdir(raw_videos_root)
                        if os.path.isdir(os.path.join(raw_videos_root, f))]
    
    if not sentence_folders:
        print("❌ No sentence folders found!")
        return None, None
    
    print(f"📁 Found {len(sentence_folders)} sentences.")
    
    # Sort for consistency
    sentence_folders.sort()
    
    for sentence_idx, sentence in enumerate(sentence_folders):
        sentence_path = os.path.join(raw_videos_root, sentence)
        video_files = [f for f in os.listdir(sentence_path)
                       if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        
        # Sort videos for consistency
        video_files.sort()
        
        print(f"\n{'='*50}")
        print(f"🎬 Sentence {sentence_idx+1}/{len(sentence_folders)}: {sentence}")
        print(f"   Videos: {len(video_files)}")
        
        if test_mode:
            video_files = video_files[:2]  # Take first 2 videos in test mode
            print("⚙️ TEST MODE: Only 2 videos per sentence")
        
        for video_file in video_files:
            video_path = os.path.join(sentence_path, video_file)
            
            # Parse filename (signer_XX_rep_XX)
            filename_parts = video_file.replace('.mp4', '').replace('.mov', '').split('_')
            
            # Handle different naming patterns
            if len(filename_parts) >= 4 and 'signer' in filename_parts[0].lower():
                signer_id = filename_parts[1] if len(filename_parts) > 1 else "unknown"
                rep_number = filename_parts[3] if len(filename_parts) > 3 else "01"
            else:
                # Fallback for simple names
                signer_id = "unknown"
                rep_number = "01"
            
            try:
                # Process video
                landmarks_seq = process_single_video(video_path, holistic, target_frames)
                
                if landmarks_seq and len(landmarks_seq) > 0:
                    # Create safe filename (remove special characters)
                    safe_sentence = sentence.replace(' ', '_').replace('/', '_')
                    output_filename = f"{safe_sentence}_signer_{signer_id}_rep_{rep_number}.npy"
                    output_path = os.path.join(output_folder, output_filename)
                    
                    # Save landmarks
                    np.save(output_path, landmarks_seq)
                    
                    # Calculate coverage statistics
                    left_hand_frames = sum(1 for f in landmarks_seq if f['left_hand'] is not None)
                    right_hand_frames = sum(1 for f in landmarks_seq if f['right_hand'] is not None)
                    lip_frames = sum(1 for f in landmarks_seq if f['lip_roi'] is not None)
                    pose_frames = sum(1 for f in landmarks_seq if f['pose'] is not None)
                    total_frames = len(landmarks_seq)
                    
                    metadata.append({
                        'landmarks_file': output_filename,
                        'sentence': sentence,
                        'sentence_id': f"S{str(sentence_idx+1).zfill(3)}",
                        'signer_id': signer_id,
                        'rep_number': rep_number,
                        'original_video': video_file,
                        'total_frames': total_frames,
                        'left_hand_frames': left_hand_frames,
                        'right_hand_frames': right_hand_frames,
                        'lip_frames': lip_frames,
                        'pose_frames': pose_frames,
                        'left_hand_coverage': (left_hand_frames / total_frames) * 100,
                        'right_hand_coverage': (right_hand_frames / total_frames) * 100,
                        'lip_coverage': (lip_frames / total_frames) * 100,
                        'pose_coverage': (pose_frames / total_frames) * 100,
                        'success': True
                    })
                    print(f"   💾 Saved → {output_filename}")
                else:
                    print(f"   ⚠️ No landmarks in {video_file}")
                    
            except Exception as e:
                print(f"   ❌ Error processing {video_file}: {str(e)}")
                metadata.append({
                    'landmarks_file': 'FAILED',
                    'sentence': sentence,
                    'signer_id': signer_id,
                    'rep_number': rep_number,
                    'original_video': video_file,
                    'error': str(e),
                    'success': False
                })
    
    # Save metadata
    if metadata:
        metadata_df = pd.DataFrame(metadata)
        meta_csv = '/content/metadata/sentence_dataset_metadata.csv'
        metadata_df.to_csv(meta_csv, index=False, encoding='utf-8-sig')
        
        # Create sentence mapping
        successful_df = metadata_df[metadata_df['success']]
        sentence_mapping = {}
        for idx, sentence in enumerate(successful_df['sentence'].unique()):
            sentence_mapping[sentence] = f"S{str(idx+1).zfill(3)}"
        
        with open('/content/metadata/sentence_mapping.json', 'w', encoding='utf-8') as f:
            json.dump(sentence_mapping, f, indent=2, ensure_ascii=False)
        
        print(f"\n📊 Metadata saved → {meta_csv}")
        print(f"📊 Sentence mapping saved → /content/metadata/sentence_mapping.json")
        
        return metadata_df, sentence_mapping
    else:
        print("❌ No videos processed successfully!")
        return None, None


In [ ]:
# --- 8: Dataset Analysis ---

def analyze_sentence_dataset(metadata_df):
    """Print statistics about processed dataset"""
    print("\n" + "="*60)
    print("📊 SENTENCE DATASET ANALYSIS")
    print("="*60)
    
    # Filter successful
    success_df = metadata_df[metadata_df['success']]
    
    total = len(metadata_df)
    success = len(success_df)
    print(f"\n📈 Overall Statistics:")
    print(f"   Total videos: {total}")
    print(f"   Successfully processed: {success} ({success/total*100:.1f}%)")
    print(f"   Failed: {total - success} ({(total-success)/total*100:.1f}%)")
    
    print(f"\n📝 Sentences: {len(success_df['sentence'].unique())}")
    print(f"👥 Signers: {len(success_df['signer_id'].unique())}")
    
    print(f"\n📊 Coverage Averages:")
    print(f"   Left Hand:  {success_df['left_hand_coverage'].mean():.1f}%")
    print(f"   Right Hand: {success_df['right_hand_coverage'].mean():.1f}%")
    print(f"   Lips:       {success_df['lip_coverage'].mean():.1f}%")
    print(f"   Pose:       {success_df['pose_coverage'].mean():.1f}%")
    
    print(f"\n📝 Videos per sentence:")
    sentence_stats = success_df['sentence'].value_counts()
    for s, c in sentence_stats.items():
        print(f"   {s[:30]}{'...' if len(s)>30 else ''}: {c} videos")
    
    print(f"\n👥 Videos per signer:")
    signer_stats = success_df['signer_id'].value_counts()
    for s, c in signer_stats.items():
        print(f"   Signer {s}: {c} videos")
    
    return success_df


In [ ]:
# --- 9: Visualize Coverage ---

def visualize_coverage(metadata_df):
    """Visualize how well landmarks were detected"""
    
    success_df = metadata_df[metadata_df['success']]
    
    if len(success_df) == 0:
        print("❌ No successful videos to visualize")
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Dataset Coverage Analysis', fontsize=16)
    
    # 1. Hand Coverage Histogram
    axes[0,0].hist([success_df['left_hand_coverage'], 
                    success_df['right_hand_coverage']], 
                   bins=20, alpha=0.7, label=['Left Hand', 'Right Hand'])
    axes[0,0].set_title('Hand Coverage Distribution')
    axes[0,0].set_xlabel('Coverage %')
    axes[0,0].set_ylabel('Number of Videos')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Lip Coverage
    axes[0,1].hist(success_df['lip_coverage'], bins=20, color='red', alpha=0.7)
    axes[0,1].set_title('Lip Coverage Distribution')
    axes[0,1].set_xlabel('Coverage %')
    axes[0,1].set_ylabel('Number of Videos')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Pose Coverage
    axes[0,2].hist(success_df['pose_coverage'], bins=20, color='green', alpha=0.7)
    axes[0,2].set_title('Pose Coverage Distribution')
    axes[0,2].set_xlabel('Coverage %')
    axes[0,2].set_ylabel('Number of Videos')
    axes[0,2].grid(True, alpha=0.3)
    
    # 4. Videos per Sentence
    sentence_counts = success_df['sentence'].value_counts().head(10)
    axes[1,0].barh(range(len(sentence_counts)), sentence_counts.values)
    axes[1,0].set_yticks(range(len(sentence_counts)))
    axes[1,0].set_yticklabels([s[:20] + '...' for s in sentence_counts.index])
    axes[1,0].set_title('Top 10 Sentences by Video Count')
    axes[1,0].set_xlabel('Number of Videos')
    
    # 5. Videos per Signer
    signer_counts = success_df['signer_id'].value_counts()
    axes[1,1].bar(signer_counts.index, signer_counts.values)
    axes[1,1].set_title('Videos per Signer')
    axes[1,1].set_xlabel('Signer ID')
    axes[1,1].set_ylabel('Number of Videos')
    
    # 6. Success Rate Pie
    success_count = len(success_df)
    fail_count = len(metadata_df) - success_count
    axes[1,2].pie([success_count, fail_count], 
                  labels=['Success', 'Failed'],
                  autopct='%1.1f%%',
                  colors=['green', 'red'])
    axes[1,2].set_title('Processing Success Rate')
    
    plt.tight_layout()
    plt.savefig('/content/visualizations/coverage_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ Visualization saved to /content/visualizations/coverage_analysis.png")


In [ ]:
# --- 10: Verify Landmark Files ---

def verify_landmark_files(folder='/content/landmarks_data'):
    """Check a few saved .npy files"""
    print("\n🔍 Verifying sample landmark files...")
    files = [f for f in os.listdir(folder) if f.endswith('.npy')]
    
    if not files:
        print("❌ No landmark files found!")
        return
    
    print(f"Found {len(files)} landmark files.")
    
    for file in files[:3]:  # Check first 3
        try:
            data = np.load(os.path.join(folder, file), allow_pickle=True)
            print(f"\n📄 {file}")
            print(f"   Shape: {data.shape}")
            print(f"   Frames: {len(data)}")
            if len(data) > 0:
                print(f"   Keys in first frame: {list(data[0].keys())}")
                
                # Check data types
                for key in ['left_hand', 'right_hand', 'lip_roi']:
                    if data[0][key] is not None:
                        print(f"   {key} length: {len(data[0][key])}")
        except Exception as e:
            print(f"   ❌ Error loading {file}: {e}")


In [ ]:
# --- 11: Export Dataset Summary ---

def export_dataset_summary(metadata_df):
    """Export a human-readable summary"""
    
    success_df = metadata_df[metadata_df['success']]
    
    summary = {
        'total_videos': len(metadata_df),
        'successful_videos': len(success_df),
        'failed_videos': len(metadata_df) - len(success_df),
        'success_rate': f"{len(success_df)/len(metadata_df)*100:.1f}%",
        'num_sentences': len(success_df['sentence'].unique()),
        'num_signers': len(success_df['signer_id'].unique()),
        'average_coverage': {
            'left_hand': f"{success_df['left_hand_coverage'].mean():.1f}%",
            'right_hand': f"{success_df['right_hand_coverage'].mean():.1f}%",
            'lip': f"{success_df['lip_coverage'].mean():.1f}%",
            'pose': f"{success_df['pose_coverage'].mean():.1f}%"
        },
        'sentences': {}
    }
    
    # Add per-sentence details
    for sentence in success_df['sentence'].unique():
        sentence_videos = success_df[success_df['sentence'] == sentence]
        summary['sentences'][sentence] = {
            'video_count': len(sentence_videos),
            'signers': list(sentence_videos['signer_id'].unique()),
            'avg_left_hand': f"{sentence_videos['left_hand_coverage'].mean():.1f}%"
        }
    
    # Save as JSON
    with open('/content/metadata/dataset_summary.json', 'w', encoding='utf-8') as f:
        json.dump(summary, f, indent=2, ensure_ascii=False)
    
    # Save as text report
    with open('/content/metadata/dataset_report.txt', 'w', encoding='utf-8') as f:
        f.write("="*60 + "\n")
        f.write("SLSL MEDICAL DATASET REPORT\n")
        f.write("="*60 + "\n\n")
        f.write(f"Generated on: {pd.Timestamp.now()}\n\n")
        f.write(f"Total Videos: {summary['total_videos']}\n")
        f.write(f"Successful: {summary['successful_videos']} ({summary['success_rate']})\n")
        f.write(f"Failed: {summary['failed_videos']}\n\n")
        f.write(f"Number of Sentences: {summary['num_sentences']}\n")
        f.write(f"Number of Signers: {summary['num_signers']}\n\n")
        f.write("Average Coverage:\n")
        for k, v in summary['average_coverage'].items():
            f.write(f"  {k}: {v}\n")
    
    print("✅ Dataset summary exported to /content/metadata/")
    return summary


In [ ]:
# --- 12: Main Pipeline ---

def main_pipeline(test_mode=True):
    print("\n" + "="*60)
    print("🚀 SLSL SENTENCE-BASED DATASET PIPELINE")
    print("="*60)
    
    # Step 1: Folder setup
    print("\n📁 Step 1: Creating folders...")
    create_sentence_based_structure()
    
    # Step 2: Copy dataset from Google Drive
    print("\n📂 Step 2: Copying dataset from Drive...")
    # !!! IMPORTANT: CHANGE THIS PATH TO MATCH YOUR FOLDER !!!
    drive_path = '/content/drive/MyDrive/SLSL_Medical_Dataset'
    sentences = copy_from_drive(drive_path=drive_path)
    
    if not sentences:
        print("\n❌ No dataset found! Please check your Drive path.")
        print("Available folders in MyDrive:")
        !ls /content/drive/MyDrive/
        return None, None
    
    # Step 3: Process dataset
    print("\n🎯 Step 3: Processing videos...")
    print(f"{'='*50}")
    metadata_df, sentence_mapping = process_sentence_based_dataset(
        test_mode=test_mode,
        target_frames=60
    )
    
    # Step 4: Analyze and verify
    if metadata_df is not None and len(metadata_df) > 0:
        print("\n📊 Step 4: Analyzing results...")
        success_df = analyze_sentence_dataset(metadata_df)
        
        print("\n🔍 Step 5: Verifying landmark files...")
        verify_landmark_files()
        
        print("\n📈 Step 6: Generating visualizations...")
        visualize_coverage(metadata_df)
        
        print("\n📋 Step 7: Exporting summary...")
        export_dataset_summary(metadata_df)
        
        print("\n" + "="*60)
        print("🎉 DATASET CREATION COMPLETE!")
        print("="*60)
        print("\n✅ Output files saved in:")
        print("   📁 /content/landmarks_data/ - Landmark .npy files")
        print("   📁 /content/metadata/ - CSV, JSON, and reports")
        print("   📁 /content/visualizations/ - Charts and graphs")
        
        return metadata_df, sentence_mapping
    else:
        print("\n❌ Dataset creation failed. No videos processed.")
        return None, None


In [ ]:
# --- 13: ZIP and Download Results ---\

def zip_and_download():
    """Zip all results for download"""
    print("\n📦 Creating download package...")
    
    # Create zip file
    !zip -r /content/slsl_dataset_results.zip \
        /content/landmarks_data \
        /content/metadata \
        /content/visualizations
    
    print("✅ Results zipped to: /content/slsl_dataset_results.zip")
    
    # Download (will prompt in Colab)
    from google.colab import files
    files.download('/content/slsl_dataset_results.zip')


In [ ]:
# --- 14: RUN THE PIPELINE ---

print("\n" + "="*60)
print("🏁 READY TO RUN")
print("="*60)

# Choose mode:
# test_mode=True  : Process 2 videos per sentence (fast, for testing)
# test_mode=False : Process ALL videos (full dataset)

# For TESTING (recommended first):
print("\n🔧 TEST MODE: Processing 2 videos per sentence")
metadata, mapping = main_pipeline(test_mode=True)

# For FULL DATASET (uncomment when ready):
# print("\n🚀 FULL MODE: Processing ALL videos")
# metadata, mapping = main_pipeline(test_mode=False)


In [ ]:
# --- 15: Check Results ---

if metadata is not None:
    print("\n" + "="*60)
    print("📋 QUICK RESULTS SUMMARY")
    print("="*60)
    
    success_df = metadata[metadata['success']]
    print(f"\n✅ Successfully processed: {len(success_df)} videos")
    print(f"📝 Sentences: {len(success_df['sentence'].unique())}")
    print(f"👥 Signers: {len(success_df['signer_id'].unique())}")
    
    print("\n📊 First 5 entries in metadata:")
    print(success_df[['sentence', 'signer_id', 'left_hand_coverage', 
                     'right_hand_coverage', 'lip_coverage']].head())
else:
    print("\n❌ No results to show. Run the pipeline first.")


In [ ]:
# --- 16: Download Results (Optional) ---

# Uncomment the line below to download all results as a zip file
# zip_and_download()

print("\n" + "="*60)
print("🏁 END OF NOTEBOOK")
print("="*60)